# Getting Started with Labellerr SDK

This notebook demonstrates how to use the Labellerr SDK for video processing and scene detection. The SDK provides powerful tools for managing video datasets, processing videos, and detecting scene changes using various algorithms.

### Import the required Classes from Labellerr SDK
We'll start by importing the essential classes needed for working with the SDK:

In [1]:
from labellerr.client import LabellerrClient
from labellerr.core.datasets import LabellerrVideoDataset
import os

## Authentication Setup

Before using the Labellerr SDK, you need to set up your authentication credentials. These credentials ensure secure access to the Labellerr platform and its services.

### Required Credentials:

1. **API Key & API Secret**
   - Log in to your Labellerr account
   - Navigate to the "Get API" tab
   - Copy your unique API key and secret

2. **Client ID**
   - This is a unique identifier for your application
   - Contact Labellerr support to obtain your client ID
   
⚠️ Important: Never share these credentials or commit them to version control.

In [2]:
api_key = ""
api_secret = ""
client_id = ""

## Project Configuration

### Dataset and Project IDs
To work with specific datasets and projects in Labellerr, you need their respective IDs. These IDs are unique identifiers that link your code to the correct resources on the platform.

How to obtain the IDs:
1. Go to the Labellerr platform
2. Create or select an existing dataset
3. Create or select an existing project
4. Copy the dataset_id and project_id from their respective pages

Note: The dataset_id is a UUID format string, while the project_id is typically a human-readable string.

In [3]:
# go to our platform to create dataset and project then get their ids
dataset_id = "16257fd6-b91b-4d00-a680-9ece9f3f241c"
project_id = "gabrila_artificial_duck_74237"

## Initializing the Labellerr SDK

### Create LabellerrClient Instance
Now we'll create instances of the main SDK classes:

1. **LabellerrClient**: The main client that handles communication with the Labellerr API
2. **LabellerrVideoDataset**: A specialized class for working with video datasets

These instances will be used for all subsequent operations with the platform.

In [4]:
client = LabellerrClient(api_key, api_secret, client_id) 
dataset = LabellerrVideoDataset(client, dataset_id, project_id)

In [5]:
results = dataset.process_all_videos()


######################################################################
# Starting batch video processing for dataset: 16257fd6-b91b-4d00-a680-9ece9f3f241c
######################################################################

Total file IDs extracted: 1

Creating LabellerrFile instances for 1 files...
Successfully created 1 LabellerrFile instances

Processing 1 video files...


Starting download of 1 files...

Processing file: c44f38f6-0186-436f-8c2d-ffb50a539c76

[1/4] Fetching frame data from API (0 to 1440)...
Retrieved 1440 frames

[2/4] Setting up output folders...

[3/4] Downloading frames...
Starting download of 1440 frames...
Frames downloaded: 1440/1440 (1440 successful, 0 failed)

[4/4] Creating video from frames...
Running command: ffmpeg -y -start_number 0 -framerate 30 -i ./Labellerr_datastets\16257fd6-b91b-4d00-a680-9ece9f3f241c\c44f38f6-0186-436f-8c2d-ffb50a539c76\%d.jpg -c:v libx264 -pix_fmt yuv420p ./Labellerr_datastets\16257fd6-b91b-4d00-a680-9ece9f3f241c\c44f38f6-0

### Processing Videos
The `process_all_videos()` method will:
- Fetch all videos in the dataset
- Process them according to the configured settings
- Return the results of the processing

This is typically used as the first step in video analysis to ensure all videos are properly prepared for further processing.

## Scene Detection

### Available Scene Detection Methods
Labellerr SDK provides multiple algorithms for scene detection in videos:

1. **PySceneDetect**: 
   - Python-based scene detection
   - Uses content-aware detection
   - Good for general-purpose scene detection

2. **SSIMSceneDetect**:
   - Uses Structural Similarity Index (SSIM)
   - Better for detecting subtle scene changes
   - More computationally intensive but more accurate

3. **FFMPEGSceneDetect**:
   - Uses FFMPEG for scene detection
   - Fastest method
   - Good for quick analysis of large video files

Choose the method that best suits your needs based on accuracy requirements and processing speed constraints.

In [15]:
from labellerr.services.video_sampling.pyscene_detect import PySceneDetect
from labellerr.services.video_sampling.ssim import SSIMSceneDetect
from labellerr.services.video_sampling.ffmpeg import FFMPEGSceneDetect

c:\Users\HP\.conda\envs\SDk\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Scene Detection Implementation

### Setting up the Scene Detector
Now we'll set up the scene detection process:

1. First, we'll define the dataset directory where our videos are stored
2. Then we'll create an instance of our chosen detector
3. Finally, we'll process each video in the dataset

Note: Make sure you have sufficient disk space for storing the extracted scenes, as this process can generate multiple files per video.

In [11]:
dataset_dir = f".\Labellerr_datastets\{dataset_id}"

In [18]:
detector = FFMPEGSceneDetect()

### Initialize the Scene Detector
Here we create an instance of the SSIMSceneDetect class. This detector uses the Structural Similarity Index Measure (SSIM) to identify scene changes in videos. SSIM is particularly effective at detecting subtle changes between frames.

In [19]:
for filename in os.listdir(dataset_dir):
    file_path = os.path.join(dataset_dir, filename)
    
    if os.path.isfile(file_path):
        detector.detect_and_extract(file_path)

Keyframes extracted to FFMPEG_detects\16257fd6-b91b-4d00-a680-9ece9f3f241c\c44f38f6-0186-436f-8c2d-ffb50a539c76\frames
JSON mapping saved to: FFMPEG_detects\16257fd6-b91b-4d00-a680-9ece9f3f241c\c44f38f6-0186-436f-8c2d-ffb50a539c76\c44f38f6-0186-436f-8c2d-ffb50a539c76_mapping.json


### Process Videos for Scene Detection

The following code block:
1. Iterates through all files in the dataset directory
2. Constructs the full file path for each video
3. Verifies that each path points to a file (not a directory)
4. Applies scene detection to each video using the `detect_and_extract` method

The detected scenes will be saved in a subdirectory with the same name as the input video file. Each scene will be saved as a separate video file.